In [ ]:
!git clone https://github.com/emilwallner/Coloring-greyscale-images-in-Keras

Cloning into 'Coloring-greyscale-images-in-Keras'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 1141 (delta 2), reused 0 (delta 0), pack-reused 1135
Receiving objects: 100% (1141/1141), 14.43 MiB | 32.48 MiB/s, done.
Resolving deltas: 100% (495/495), done.


In [ ]:
  from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
  from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
  from keras.layers.normalization import BatchNormalization
  from keras.callbacks import TensorBoard
  from keras.models import Sequential
  from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
  from skimage.color import rgb2lab, lab2rgb, rgb2gray
  from skimage.io import imsave
  import numpy as np
  import os
  import random
  import tensorflow as tf

In [ ]:
# Get images 
X = [] 
for filename in os.listdir('Coloring-greyscale-images-in-Keras/Full-version/Train/'):
    X.append(img_to_array(load_img('Coloring-greyscale-images-in-Keras/Full-version/Train/'+filename))) 
X = np.array(X, dtype=float) 

# Set up training and test data 
split = int(0.95*len(X)) 
Xtrain = X[:split] 
Xtrain = 1.0/255*Xtrain



In [ ]:
#Design the neural network 
model = Sequential() 
model.add(InputLayer(input_shape=(256, 256, 1))) 
model.add(Conv2D(64, (3, 3), activation='relu', padding='same')) 
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2)) 
model.add(Conv2D(128, (3, 3), activation='relu', padding='same')) 
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2)) 
model.add(Conv2D(256, (3, 3), activation='relu', padding='same')) 
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2)) 
model.add(Conv2D(512, (3, 3), activation='relu', padding='same')) 
model.add(Conv2D(256, (3, 3), activation='relu', padding='same')) 
model.add(Conv2D(128, (3, 3), activation='relu', padding='same')) 
model.add(UpSampling2D((2, 2))) 
model.add(Conv2D(64, (3, 3), activation='relu', padding='same')) 
model.add(UpSampling2D((2, 2))) 
model.add(Conv2D(32, (3, 3), activation='relu', padding='same')) 
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same')) 
model.add(UpSampling2D((2, 2)))



In [ ]:
# Finish model 
model.compile(optimizer='rmsprop', loss='mse') 
# Image transformer 
datagen = ImageDataGenerator(
    shear_range=0.2, 
    zoom_range=0.2, 
    rotation_range=20, 
    horizontal_flip=True)



In [ ]:
# Generate training data 
batch_size = 50 
def image_a_b_gen(batch_size): 
    for batch in datagen.flow(Xtrain, batch_size=batch_size): 
        lab_batch = rgb2lab(batch) 
        X_batch = lab_batch[:,:,:,0] 
        Y_batch = lab_batch[:,:,:,1:] / 128 
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)



In [ ]:
# Train model 
TensorBoard(log_dir='/output') 
model.fit_generator(image_a_b_gen(batch_size), steps_per_epoch=10000, epochs=1)



  277/10000 [..............................] - ETA: 23:41:35 - loss: 0.0074

KeyboardInterrupt: ignored

In [ ]:
# Test images 
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0] 
Xtest = Xtest.reshape(Xtest.shape+(1,)) 
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:] 
Ytest = Ytest / 128 
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))



1/1 [==============================] - 0s 2ms/step - loss: 0.0372
0.03716125711798668


In [ ]:
# Load black and white images 
color_me = [] 
for filename in os.listdir('Coloring-greyscale-images-in-Keras/Full-version/Test/'): 
    color_me.append(img_to_array(load_img('Coloring-greyscale-images-in-Keras/Full-version/Test/'+filename))) 
color_me = np.array(color_me, dtype=float) 
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0] 
color_me = color_me.reshape(color_me.shape+(1,))



In [ ]:
# Test model 
output = model.predict(color_me) 
output = output * 128



In [ ]:
# Output colorizations 
for i in range(len(output)): 
    cur = np.zeros((256, 256, 3)) 
    cur[:,:,0] = color_me[i][:,:,0] 
    cur[:,:,1:] = output[i] 
    imsave("result/img_"+str(i)+".png", lab2rgb(cur))

